In [1]:
#declare libraries
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import datetime
from dateutil.relativedelta import relativedelta #DOWNLOAD THIS PACKAGE#
import os
import requests #DOWNLOAD THIS PACKAGE#
import zipfile #DOWNLOAD THIS PACKAGE#
import io #DOWNLOAD THIS PACKAGE#
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import os
import time
import pylab as pl
import scipy.stats
from shapely.geometry import Point
import statsmodels.formula.api as smf
import geopandas as gpd
import random
import json
from fiona.crs import from_epsg
import pysal as ps
import math
import matplotlib.pylab as plt
import seaborn as sbn
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
import statsmodels.api as sm

ImportError: libjpeg.so.8: cannot open shared object file: No such file or directory

In [2]:
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")
    puidata = os.getenv("PUIDATA")

In [4]:
#i got these links from http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml
# essentially these are just the trip level data for hire vehicles, green and yellow taxi datasets for June 2018
#download data and place them into the PUIdata folder
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2018-06.csv > ~/PUIdata/fhv_tripdata_2018-06.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2018-06.csv > ~/PUIdata/green_tripdata_2018-06.csv
!curl https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-06.csv > ~/PUIdata/yellow_tripdata_2018-06.csv

#lookup table related to taxi zones
!curl https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv > ~/PUIdata/taxi+_zone_lookup.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1366M  100 1366M    0     0  14.2M      0  0:01:36  0:01:36 --:--:-- 12.7M     0  0:01:32  0:01:24  0:00:08 13.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.1M  100 63.1M    0     0  12.2M      0  0:00:05  0:00:05 --:--:-- 12.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  733M  100  733M    0     0  12.2M      0  0:00:59  0:00:59 --:--:-- 12.4M   0     0  10.2M      0  0:01:11  0:00:25  0:00:46 10.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12322  100 12322    0     0   1

In [5]:
#fix to remove empty line at line 2 of yellow_trip_data
#!sed -i '2d' ~/PUIdata/yellow_tripdata_2018-06.csv

In [6]:
#download shapefiles related to taxizones 
def dl_unzip(url,output_path):
    '''
    description: downloads a zipfile, given a url address and unpacks it at provided location
    ----------------------------------------------------------------------------------
    Parameters - Parameter Type - Description - Example input
    1.) url - <string> - refers url address for the zipfile - "http://www.lalaland.com/goose.zip"
    2.) output_path - <string> - refers to the working directory where you want to unpack your file - "~\sebs_bar"
    ----------------------------------------------------------------------------------
    room for improvement
    1.) error handling
    2.) returns status
    
    has dependencies on the following packages
    requests
    zipfile
    ----------------------------------------------------------------------------------
    Example usage:
    dl_unzip("http://www.lalaland.com/goose.zip","~/sebs_bar")
    '''    
    r = requests.get(url, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(output_path)
    
dl_unzip("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip",puidata)

In [3]:
yellow = pd.read_csv('~/PUIdata/yellow_tripdata_2018-06.csv')
green = pd.read_csv('~/PUIdata/green_tripdata_2018-06.csv')
FHV = pd.read_csv('~/PUIdata/fhv_tripdata_2018-06.csv')

In [4]:
lookup_id = pd.read_csv('~/PUIdata/taxi+_zone_lookup.csv')

In [5]:
lookup_id.head()

LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone

In [6]:
borough = dict(zip(lookup_id['LocationID'],lookup_id['Borough']))
zone = dict(zip(lookup_id['LocationID'],lookup_id['Zone']))

In [6]:
yellow.head()

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2018-06-01 00:15:40   2018-06-01 00:16:46                1   
1         1  2018-06-01 00:04:18   2018-06-01 00:09:18                1   
2         1  2018-06-01 00:14:39   2018-06-01 00:29:46                1   
3         1  2018-06-01 00:51:25   2018-06-01 00:51:29                3   
4         1  2018-06-01 00:55:06   2018-06-01 00:55:10                1   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0            0.0           1                  N           145           145   
1            1.0           1                  N           230           161   
2            3.3           1                  N           100           263   
3            0.0           1                  N           145           145   
4            0.0           1                  N           145           145   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0             2          3.0    0.5      0.5        0.00           0.0   
1             1          5.5    0.5      0.5        1.35           0.0   
2             2         13.0    0.5      0.5        0.00           0.0   
3             2          2.5    0.5      0.5        0.00           0.0   
4             2          2.5    0.5      0.5        0.00           0.0   

   improvement_surcharge  total_amount  
0                    0.3          4.30  
1                    0.3          8.15  
2                    0.3         14.30  
3                    0.3          3.80  
4                    0.3          3.80

In [12]:
green.head()

VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2018-06-01 00:33:55   2018-06-01 00:36:13                  N   
1         2  2018-06-01 00:40:36   2018-06-01 00:49:46                  N   
2         2  2018-06-01 00:57:12   2018-06-01 01:02:58                  N   
3         2  2018-06-01 00:10:13   2018-06-01 00:16:27                  N   
4         1  2018-06-01 00:32:08   2018-06-01 00:52:06                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0           1            66            33                5           0.51   
1           1            25            49                5           1.97   
2           1            61            49                5           1.40   
3           1            49            97                1           1.36   
4           1            75           127                1           7.90   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0          4.0    0.5      0.5        0.70           0.0        NaN   
1          9.0    0.5      0.5        2.06           0.0        NaN   
2          6.5    0.5      0.5        0.00           0.0        NaN   
3          7.0    0.5      0.5        0.00           0.0        NaN   
4         24.0    0.5      0.5        6.30           0.0        NaN   

   improvement_surcharge  total_amount  payment_type  trip_type  
0                    0.3          6.00             1          1  
1                    0.3         12.36             1          1  
2                    0.3          7.80             2          1  
3                    0.3          8.30             2          1  
4                    0.3         31.60             1          1

In [13]:
FHV.head()

Dispatching_base_num      Pickup_DateTime     DropOff_datetime  \
0               B00628  2018-06-11 00:10:20  2018-06-11 01:13:39   
1               B00628  2018-06-11 05:12:50  2018-06-11 06:53:50   
2               B00628  2018-06-11 05:14:16  2018-06-11 06:21:59   
3               B00628  2018-06-11 05:15:23  2018-06-11 06:27:47   
4               B00628  2018-06-11 06:36:46  2018-06-11 07:34:50   

   PUlocationID  DOlocationID  SR_Flag  
0          88.0         200.0      NaN  
1          95.0         132.0      NaN  
2         142.0         132.0      NaN  
3         236.0         132.0      NaN  
4         158.0           1.0      NaN

## get only the necessary columns from each dataset. For this assignment i only need the pickup-dropoff pairs for each vehicle type. 

In [7]:
drop_cols=[x for x in yellow.columns.get_values() if x not in ['PULocationID','DOLocationID']]
yellow=yellow.drop(drop_cols,axis=1)

In [8]:
drop_cols=[x for x in green.columns.get_values() if x not in ['PULocationID','DOLocationID']]
green=green.drop(drop_cols,axis=1)

In [9]:
drop_cols=[x for x in FHV.columns.get_values() if x not in ['PUlocationID','DOlocationID']]
FHV=FHV.drop(drop_cols,axis=1)

In [17]:
yellow.head()

PULocationID  DOLocationID
0           145           145
1           230           161
2           100           263
3           145           145
4           145           145

In [18]:
green.head()

PULocationID  DOLocationID
0            66            33
1            25            49
2            61            49
3            49            97
4            75           127

In [19]:
FHV.head()

PUlocationID  DOlocationID
0          88.0         200.0
1          95.0         132.0
2         142.0         132.0
3         236.0         132.0
4         158.0           1.0

In [10]:
#dropnas
yellow.dropna(inplace=True)
green.dropna(inplace=True)
FHV.dropna(inplace=True)

In [11]:
#fix FHV's PUlocationID, DOlocationID
FHV['PUlocationID'] = FHV['PUlocationID'].astype(int) 
FHV['DOlocationID'] = FHV['DOlocationID'].astype(int)
FHV.rename(columns={"PUlocationID":"PULocationID","DOlocationID":"DOLocationID"},inplace=True)

In [11]:
FHV.head()

PULocationID  DOLocationID
0            88           200
1            95           132
2           142           132
3           236           132
4           158             1

In [12]:
datalist = [yellow,green,FHV]
columnlist = ['PULocationID','DOLocationID']
dictlist = ["borough","zone"]
init = 0
for i in datalist:
    for j in columnlist:
            for x in dictlist:
                i[j+"_"+x] = i[j].map(eval(x))

In [13]:
yellow.head()

PULocationID  DOLocationID PULocationID_borough  \
0           145           145               Queens   
1           230           161            Manhattan   
2           100           263            Manhattan   
3           145           145               Queens   
4           145           145               Queens   

                PULocationID_zone DOLocationID_borough  \
0  Long Island City/Hunters Point               Queens   
1       Times Sq/Theatre District            Manhattan   
2                Garment District            Manhattan   
3  Long Island City/Hunters Point               Queens   
4  Long Island City/Hunters Point               Queens   

                DOLocationID_zone  
0  Long Island City/Hunters Point  
1                  Midtown Center  
2                  Yorkville West  
3  Long Island City/Hunters Point  
4  Long Island City/Hunters Point

### load the shapefile

In [14]:
taxishp = gpd.GeoDataFrame.from_file(puidata+"/"+"taxi_zones.shp")

NameError: name 'gpd' is not defined